# Yolov3 Dataloader (Google Colab) V2

Author: Kevin Yu \
Email: kevinyu211@yahoo.com \
Site: www.hikariai.net \
 \
** This tool is tailored for those who want to train their custom dataset on a Yolov3 Model. 

** Make sure you read the instructions for each step carefully !!!

** If you following the instructions below step by step, it will generate a new trained-weight in the end, and you may download it to your local machine.

** In the github repo, you can find a "demo.sh" to test the result. 




#### Good Luck!

# Check GPU

In [0]:
# Check if you get a P4 or a P100 Nvidia GPU
# If you get a K80, you may reset the runtime

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

 # Setup

### Install cuDNN-10.0 and other dependecies

In [0]:
# Download the cuDNN driver
!wget https://objectstorage.ca-toronto-1.oraclecloud.com/n/yzpqsgba6ssd/b/bucket-20200415-0121/o/cudnn-10.0-linux-x64-v7.5.0.56.tgz -O cudnn-10.0-linux-x64-v7.5.0.56.tgz
# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# install python-nvcc plugin
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

# check if installed successfully
!/usr/local/cuda/bin/nvcc --version

### Clone Yolov3-Darknet

In [0]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!wget https://objectstorage.ca-toronto-1.oraclecloud.com/n/yzpqsgba6ssd/b/bucket-20200415-0121/o/yolov3-tiny.weights -q --show-progress --no-clobber -O /content/darknet/yolov3-tiny.weights
!wget https://objectstorage.ca-toronto-1.oraclecloud.com/n/yzpqsgba6ssd/b/bucket-20200415-0121/o/darknet53.conv.74 -q --show-progress --no-clobber -O /content/darknet/cfg/darknet53.conv.74

### Compile YOLOv3-Darknet with CUDA 10.0

In [0]:
!export PATH=/usr/local/cuda-10.0/bin${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda-10.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
%cd /content/darknet
!sed -i 's|GPU=0|GPU=1|' Makefile
!sed -i 's|CUDNN=0|CUDNN=1|' Makefile
!sed -i 's|OPENCV=0|OPENCV=1|' Makefile

In [0]:
!make

### Clone the project from Github

In [0]:
%cd /content
!git clone https://github.com/yqlbu/yolov3-dataloader-cloud-v2 yolov3-dataloader-cloud
!mkdir /content/yolov3-dataloader-cloud/backup
!mkdir /content/yolov3-dataloader-cloud/img

# Upload dataset

*** Please follow the steps below to upload your dataset to the data folder located at  '/content/data'

**Step #0: Prepare your own dataset**

- On your local machine, create a new folder named "data"
- Put all the images and the ".xml" files associated with each image into this data folder
- Zip the data folder

*** Noted: 
- You should have a zip file called "data.zip" that contains all the images (.png/.jpg/.jpeg) aligned with the .xml files to be trained before moving to the next step.

**Step #1: Mount your Google Drive**

To do so, run the code below. \
- Paste the token \
- Hit enter \
- Check if your Google Drive has been successfully mounted in Colab named "drive"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**Step #2: Unzip the data file from your Google Drive to the 'yolov3-dataloader-cloud' directory**
- Note: if your path contains a " " (space), you should add a '\ ' between characters
- eg: '/content/drive/Shared\ drives/dataset/data.zip'
- To easily find the path just simply find the data.zip file in your drive and right-click the "copy path"
- Please make sure you correctly find your file path before executing this cell. Otherwise, it won't work.

In [0]:
!unzip <DATA_PATH> -d /content/yolov3-dataloader-cloud
# eg: unzip /content/drive/My\ Drive/Colab\ Notebooks/data.zip -d /content/yolov3-dataloader-cloud
!echo '  Finished loading dataset !'

*** Noted: If the above commands excuted correctly, you may click the 'refresh button' on the left column, and you will see the data folder created inside the 'yolov3-dataloader-cloud' directory

*** Now you should be good to move forward ! Good Luck !

# Define Utils

In [0]:
# Download an image
def ImgDownload(img_url,img_name):
  import urllib.request
  print('Image downloading...')
  img_path = '/content/yolov3-dataloader-cloud/img/'
  img_name = img_name+'.jpg'
  urllib.request.urlretrieve(img_url, img_path+img_name)
  print('Image displaying...')
  imShow(img_path+img_name)

# Display an image
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# Create config files for training

*** Notes (Please read carefully before you excute the cells below):

 - Please make sure you set config parameters below corrently. Otherwise, you will not be able to activate the training process.
 - **MODEL_NAME** is the name of your new model
 - **CLASS_NAME** is associated with all the classes that your dataset contains, please fill in the CLASS_NAME list correctly with your classes
 - **CLASS_NUM** is associated with the total number of classes that your dataset contains
 - **MAX_BATCHES** is the total number of iterations in the training
 - If you set all the parameters mentioned above properly, you should be good to move forward

In [0]:
import os

#New Model Parameters

#Replace NEW_MODEL_NAME with your new model name
MODEL_NAME = 'MODEL_NAME'
#Put your classes tag below
CLASS_NAME = [
    "class_1",
    "class_2"
]
#Replace # with your classes number (int), 2 is set as the default value
CLASS_NUM = 2
#Replace your desired batch # (6000 as default)
#Ideally the MAX_BATCHES = 2500*CLASS_NUM
MAX_BATCHES = 6000

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

#Show Training process
IMG_PATH = '/content/darknet/chart.png'

%cd /content/yolov3-dataloader-cloud

In [0]:
#class.name
with open(MODEL_NAME+'.names', "a+") as file:
    file.seek(0)
    for i in CLASS_NAME:
        file.write(i + '\n')
    print(MODEL_NAME+'.names'+' has been successfully created!')
    file.close()

In [0]:
#class.data
with open(MODEL_NAME+'.data', "a+") as file:
    file.seek(0)
    file.write('classes = ' + str(CLASS_NUM) + '\n')
    file.write('train = /content/yolov3-dataloader-cloud/train.txt' + '\n')
    file.write('valid = /content/yolov3-dataloader-cloud/test.txt' + '\n')
    file.write('backup = /content/yolov3-dataloader-cloud/backup' + '\n')
    file.write('names = /content/yolov3-dataloader-cloud/' + MODEL_NAME + '.names' + '\n')
    print(MODEL_NAME+'.data'+' has been successfully created!')
    file.close()

In [0]:
#dataDispatch.py
with open('dataDispatch.py', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open('dataDispatch.py', "w") as file:
    all_lines[7] = 'classes = ' + str(CLASS_NAME) + '\n'
    print(all_lines[7].strip())
    file.writelines(all_lines)
    print('dataDispatch.py'+' has been successfully modified!')
    file.close()

In [0]:
#class-yolov3-tiny.cfg
with open('temp-config.txt', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open(MODEL_NAME+'-yolov3-tiny.cfg', "w") as file:
    #modify class #
    all_lines[134] = 'classes=' + str(CLASS_NUM) + '\n'
    all_lines[176] = 'classes=' + str(CLASS_NUM) + '\n'
    #modify max-batches
    all_lines[19] = 'max_batches = ' + str(MAX_BATCHES) + '\n'    
    #modify filter ## filter = (CLASS_NUM+5) *3 ##
    all_lines[126] = 'filters=' + str((CLASS_NUM+5) *3) + '\n'
    all_lines[170] = 'filters=' + str((CLASS_NUM+5) *3) + '\n'
    #overwrite all
    file.writelines(all_lines)
    print(MODEL_NAME+'-yolov3-tiny.cfg'+' has been successfully created!')
    file.close()

In [0]:
#train.sh
with open('train.sh', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open('train.sh', "w") as file:
    all_lines[5] = '$PRJ_PATH/' + MODEL_NAME + '.data \\' + '\n'
    all_lines[6] = '$PRJ_PATH/' + MODEL_NAME+'-yolov3-tiny.cfg' + ' \\' + '\n'
    file.writelines(all_lines)
    print('train.sh'+' has been successfully modified!')
    file.close()

# Training

*** Notes:

 - Hit the "Refresh" button on the left column, and then navigate to the "/yolov3-dataloader-cloud" directory
 - Check if the directory contains the **".data"**, the **".names"**, and the **".cfg"** files. If you miss one or more of the files, please check the instructions from the above steps.
 - Now you should be good to activate the training process, good luck !

In [0]:
#start training
!export PATH=/usr/local/cuda-10.0/bin${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda-10.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
%cd /content/yolov3-dataloader-cloud/
!python dataDispatch.py
!source train.sh
!echo '*** Congratulations! Training has completed. Enjoy! ***'

**Show the final training result**

*** You can observe a chart of how your model did throughout the training process by running the below command. It shows a chart of your average loss vs. iterations. For your model to be 'accurate' you would aim for a loss under 1.

In [0]:
#view the training result
imShow(IMG_PATH)

*** Noted:
- Once the training has finished, the final weight will be saved to the **'/content/yolov3-dataload/backup'** directory.
- We will use it for the demo.

# Demo

## Test the trained model with an image input

**Step #1: Download an Image to test your model**

In [0]:
img_url = 'IMG_URL'
# eg: img_url = 'https://tasteasianfood.com/wp-content/uploads/2016/01/et17-720x405.jpg'
img_name = 'IMG_NAME'
# eg: img_name = 'eggtart'
ImgDownload(img_url,img_name)

**Step #2: Modify the image_demo script**

In [0]:
#image_demo.sh
with open('image_demo.sh', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open('image_demo.sh', "w") as file:
    all_lines[5] = '$PRJ_PATH/' + MODEL_NAME + '.data \\' + '\n'
    all_lines[6] = '$PRJ_PATH/' + MODEL_NAME+'-yolov3-tiny.cfg' + ' \\' + '\n'
    all_lines[7] = '$PRJ_PATH/backup/' + MODEL_NAME+'-yolov3-tiny_final.weights' + ' \\' + '\n'
    all_lines[8] = '-ext_output ' + '$PRJ_PATH/img/' + img_name + '.jpg' + ' \\' + '\n'
    file.writelines(all_lines)
    print('image_demo.sh'+' has been successfully modified!')
    print('Ready to predict ' + img_name + '.jpg')
    file.close()

**Step #3: Run the Demo Script**

In [0]:
!source image_demo.sh

**Step #4: Display the prediction**

In [0]:
imShow('/content/darknet/predictions.jpg')

## Test the trained model with a video/camera input

**Step #1: Modify the demo script**

In [0]:
#video_demo.sh
with open('video_demo.sh', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open('video_demo.sh', "w") as file:
    all_lines[7] = '$PRJ_PATH/' + MODEL_NAME + '.data \\' + '\n'
    all_lines[8] = '$PRJ_PATH/' + MODEL_NAME+'-yolov3-tiny.cfg' + ' \\' + '\n'
    all_lines[9] = '$PRJ_PATH/' + MODEL_NAME+'-yolov3-tiny_final.weights' + ' \\' + '\n'
    file.writelines(all_lines)
    print('video_demo.sh'+' has been successfully modified!')
    file.close()

In [0]:
#cam_demo.sh
with open('cam_demo.sh', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open('cam_demo.sh', "w") as file:
    all_lines[7] = '$PRJ_PATH/' + MODEL_NAME + '.data \\' + '\n'
    all_lines[8] = '$PRJ_PATH/' + MODEL_NAME + '-yolov3-tiny.cfg' + ' \\' + '\n'
    all_lines[9] = '$PRJ_PATH/' + MODEL_NAME + '-yolov3-tiny_final.weights' + ' \\' + '\n'
    all_lines[10] = '-c 0 -thresh .5'
    file.writelines(all_lines)
    print('cam_demo.sh'+' has been successfully modified!')
    file.close()

In [0]:
#class.data
with open(MODEL_NAME+'.data', "w") as file:
    file.seek(0)
    file.write('classes = ' + str(CLASS_NUM) + '\n')
    file.write('train = <Folder_Path>/train.txt' + '\n')
    file.write('valid = <Folder_Path>/test.txt' + '\n')
    file.write('backup = <Folder_Path>/backup' + '\n')
    file.write('names = <Folder_Path>/' + MODEL_NAME + '.names' + '\n')
    print(MODEL_NAME+'.data'+' has been successfully created!')
    file.close()

**Step #2: Download the necessary files from Colab** \
 \
**Notes:** \
 \
** For some reasons, it is pretty hard to display the video result from Colab. You may run the script below to download all the necessary files from Colab to your local machine, and run the darknet dector from your local machine afterwards. \
** Please make sure you have installed and compiled Darknet in your local machine as well. Otherwise, you will not be able to see the predictions from a video input as expected. You may check out https://github.com/AlexeyAB/darknet for installation guide.

**Compress all necessary files**

In [0]:
!mkdir /content/custom_model

In [0]:
import os
import shutil
import zipfile

dir_src = ("/content/yolov3-dataloader-cloud")
dir_dst = ("/content/custom_model")
file_list = []
Model_Final_Weight = MODEL_NAME+'-yolov3-tiny_final.weights'
Model_Weights = MODEL_NAME+'-yolov3-tiny.cfg'

for file in os.listdir(dir_src):
    if file.endswith(".data"):
      file_list.append(file)
    if file.endswith(".names"):
      file_list.append(file)    
file_list.append('video_demo.sh')  
file_list.append('cam_demo.sh')
file_list.append(Model_Final_Weight) 
file_list.append(Model_Weights)

print("The following files are going to be exported: \n")
for file in file_list:
  print(file)
print()
print("Exporting...")

# Copy files
for filename in file_list:
  if filename.endswith('.weights'):
    shutil.copy(dir_src + '/backup/' + filename, dir_dst)
  else:
    shutil.copy(dir_src + '/' + filename, dir_dst)

# Zip target files
!zip -r /content/custom_model.zip /content/custom_model/*

print("Export done!")

**Download the zip file from Colab**

** Hit the Refresh button and you will see the custom_model.zip file \
** Right Click the custom_model.zip file and download it to your local machine.

**Step #3: Test on your local machine**

** Please make sure you have installed and compiled Darknet in your local machine as well. Otherwise, you will not be able to see the predictions from a video input as expected. You may check out https://github.com/AlexeyAB/darknet for installation guide. \
 \
** On your local machine, follow the steps below: \
- Unzip the custom_model.zip file
- Navigate to the custom_model folder
- Find the path of the folder by running $ pwd
- Open the .data file with an IDE
- Update the path of custom_model
- Open the video_demo.sh and cam_demo.sh with an IDE
- Update the path of darknet and the path of custom_model
- Place a test video file in the custom_model folder, and name it test.mp4
- Run the script
```
$ sudo chmod +x video_demo.sh cam_demo.sh
$ ./video.sh
$ ./cam.sh
```

## Notes:

- To stream the detector output, you may use the http_tcp, json, and mjpeg streaming protocol.  \
   \
  For **JSON**: add **-json_port 8080 -ext_output** to the last line of your demo scripts\
  For **mjpeg**: add **-mjpeg_port 8070 -ext_output** to the last line of your demo scripts\
     \
  Open a web browser and type localhost:8080 and you will see the stream data from yolov3 detector
  
- To store the output, you may append **-out_filename output.mp4** to the last line of your demo scripts

- If you have any suggestions or comments, please contact me via kevinyu211@yahoo.com or visit my site @ www.hikariai.net